In [1]:
!pip install wordcloud

  Obtaining dependency information for wordcloud from https://files.pythonhosted.org/packages/f5/b0/247159f61c5d5d6647171bef84430b7efad4db504f0229674024f3a4f7f2/wordcloud-1.9.3-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/300.2 kB ? eta -:--:--
   - -------------------------------------- 10.2/300.2 kB ? eta -:--:--
   ----- --------------------------------- 41.0/300.2 kB 653.6 kB/s eta 0:00:01
   ----------- --------------------------- 92.2/300.2 kB 744.7 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/300.2 kB 717.5 kB/s eta 0:00:01
   --------------- ---------------------- 122.9/300.2 kB 717.5 kB/s eta 0:00:01
   ---------------------------------------  297.0/300.2 kB 1.1 MB/s eta 0:00:01
   ---------------------------------------- 300.2/300.2 kB 1.1 MB/s eta 0:00:00


In [2]:
pip install pymongo

  Obtaining dependency information for pymongo from https://files.pythonhosted.org/packages/c8/1a/1ba5578cc3acc8ef0dfd6eb9385f9c4b0fe3039665af93bcf062c34fcdb6/pymongo-4.6.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for dnspython<3.0.0,>=1.16.0 from https://files.pythonhosted.org/packages/f6/b4/0a9bee52c50f226a3cbfb54263d02bb421c7f2adc136520729c2c689c1e5/dnspython-2.4.2-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/472.7 kB ? eta -:--:--
   ---------------------------------------- 0.0/472.7 kB ? eta -:--:--
   -- ------------------------------------- 30.7/472.7 kB ? eta -:--:--
   --- ----------------------------------- 41.0/472.7 kB 653.6 kB/s eta 0:00:01
   --- ----------------------------------- 41.0/472.7 kB 653.6 kB/s eta 0:00:01
   ----- --------------------------------- 71.7/472.7 kB 393.8 kB/s eta 0:00:02
   ----- --------------------------------- 71.7/472.7 kB 393.8 kB/s eta 0:00:02
   ------- -----------------------------

In [1]:
from wordcloud import WordCloud
import numpy as np 
import pandas as pd
from sklearn.model_selection import train_test_split
# !pip install neattext
from keras.models import load_model
import neattext.functions as nfx
import matplotlib.pyplot as plt
import plotly.express as plx
from sklearn.metrics import classification_report
import keras
from keras.layers import Embedding,Dense,LSTM,Bidirectional,GlobalMaxPooling1D,Input,Dropout
from keras.callbacks import EarlyStopping,ReduceLROnPlateau
from keras.models import Sequential
import tensorflow as tf
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer

In [2]:
from keras.utils import pad_sequences
from tqdm import tqdm
import seaborn as sns
import pickle
import warnings
warnings.filterwarnings('ignore')

In [3]:
data=pd.read_csv('new_data.txt')
data.head(20)

,Project Idea,Brainstorming Idea
0,An app for organizing outdoor team-building ac...,Mind Mapping
1,A website for DIY party decoration ideas,Random Word Association
2,A platform for virtual reality-based party exp...,Reverse Thinking
3,An AI-powered party planning assistant,Mind Mapping
4,A social media platform for sharing party play...,Random Word Association
5,A service for renting themed party decorations,Reverse Thinking
6,An app that generates party game ideas,Mind Mapping
7,A marketplace for custom-designed party invita...,Random Word Association
8,A platform for hosting virtual reality parties,Reverse Thinking
9,An online community for sharing party planning...,Mind Mapping


In [4]:
data['Brainstorming Idea'].value_counts()

Mind Mapping               427
Random Word Association     13
Reverse Thinking            12
Name: Brainstorming Idea, dtype: int64

In [5]:
data['Brainstorming Idea'].value_counts().index.values

array(['Mind Mapping', 'Random Word Association', 'Reverse Thinking'],
      dtype=object)

In [6]:
train_data,test_data=train_test_split(data,test_size=0.2,random_state=10)

In [7]:
train_data['Brainstorming Idea'].value_counts().index.values

array(['Mind Mapping', 'Reverse Thinking', 'Random Word Association'],
      dtype=object)

In [8]:
def clean_text(text):
    text_length=[]
    cleaned_text=[]
    for sent in tqdm(text):
        sent=sent.lower()
        sent=nfx.remove_special_characters(sent)
        sent=nfx.remove_stopwords(sent)
        text_length.append(len(sent.split()))
        cleaned_text.append(sent)
    return cleaned_text,text_length

In [9]:
cleaned_train_text,train_text_length=clean_text(train_data["Project Idea"])
cleaned_test_text,test_text_length=clean_text(test_data["Project Idea"])

100%|██████████| 91/91 [00:00<00:00, 92461.64it/s]


In [10]:
tokenizer=Tokenizer()
tokenizer.fit_on_texts(cleaned_train_text)

In [11]:
word_freq=pd.DataFrame(tokenizer.word_counts.items(),columns=['word','count']).sort_values(by='count',ascending=False)

In [12]:
train_text_seq=tokenizer.texts_to_sequences(cleaned_train_text)
train_text_pad=pad_sequences(train_text_seq,maxlen=100)


test_text_seq=tokenizer.texts_to_sequences(cleaned_test_text)
test_text_pad=pad_sequences(test_text_seq,maxlen=100)

In [13]:
lbl_target=LabelEncoder()
train_output=lbl_target.fit_transform(train_data['Brainstorming Idea'])
test_output=lbl_target.transform(test_data['Brainstorming Idea'])

In [14]:
with open('glove.840B.300d.pkl', 'rb') as fp:
    glove_embedding = pickle.load(fp)

In [15]:
v=len(tokenizer.word_index)
embedding_matrix=np.zeros((v+1,300), dtype=float)
for word,idx in tokenizer.word_index.items():
    embedding_vector=glove_embedding.get(word)
    if embedding_vector is not None:
        embedding_matrix[idx]=embedding_vector

In [16]:
early_stop=EarlyStopping(patience=5)
reducelr=ReduceLROnPlateau(patience=3)

In [21]:
from tensorflow import keras
from tensorflow.keras.layers import Input, Embedding, LSTM, GlobalMaxPooling1D, Dense

# Assuming you have 'v' defined as the vocabulary size and 'embedding_matrix' as the pre-trained embedding matrix

# Define the model
model = keras.Sequential([
    keras.layers.Input(shape=(100,)),
    keras.layers.Embedding(v + 1, 300, weights=[embedding_matrix], trainable=False),
    keras.layers.LSTM(20, return_sequences=True),
    keras.layers.GlobalMaxPooling1D(),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')  # Output layer with 3 units for 3 categories
])

# Compile the model
model.compile(optimizer=keras.optimizers.SGD(0.05, momentum=0.09),
              loss='sparse_categorical_crossentropy',  # Use sparse_categorical_crossentropy for multi-class classification
              metrics=['accuracy'])


In [34]:
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [29]:
svm = SVC(kernel='poly', C=1, gamma='auto')

In [36]:
svm.fit(train_text_pad,train_output)

SVC(C=1, gamma='auto', kernel='poly')

In [37]:
y_pred = svm.predict(test_text_pad)

In [38]:
accuracy = accuracy_score(test_output, y_pred)
print(f'Accuracy: {accuracy}')

Accuracy: 0.9010989010989011


In [39]:
from joblib import dump

# Save the model to a file
dump(svm, 'svm_model.joblib')

['svm_model.joblib']

In [23]:
r=model.fit(train_text_pad,train_output,validation_data=(test_text_pad,test_output),
            epochs=20,batch_size=64,callbacks=[early_stop,reducelr],)

Epoch 1/20


InvalidArgumentError: Graph execution error:

Detected at node 'sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits' defined at (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\ProgramData\Anaconda3\lib\runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "C:\ProgramData\Anaconda3\lib\site-packages\traitlets\config\application.py", line 846, in launch_instance
      app.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\ipykernel\kernelapp.py", line 712, in start
      self.io_loop.start()
    File "C:\ProgramData\Anaconda3\lib\site-packages\tornado\platform\asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "C:\ProgramData\Anaconda

In [20]:
suicide_model.save('my_model.h5')

In [35]:
suicide_model = load_model('my_model.h5')

In [36]:
def detectSuicide(text):
    cleaned_text, length = clean_text(text)
    tokens = tokenizer.texts_to_sequences(cleaned_text)
    paded = pad_sequences(tokens,maxlen=50)
    if suicide_model.predict([paded]) >=0.50:
#         print("contains suicidal thoughts")
#         print(suicide_model.predict([paded]))
        return 0
    else:
#         print("Doesn't contains suicidal thoughts")
#         print(suicide_model.predict([paded]))
        return 1
detectSuicide(["I don't want to live anymore"])

100%|██████████| 1/1 [00:00<00:00, 1004.86it/s]


1/1 [==============================] - 0s 425ms/step


0

In [37]:
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable


In [38]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from transformers import BertTokenizer, BertModel

In [39]:
bert_tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
class Transformer(nn.Module):
    def __init__(self, num_classes):
        super(Transformer, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        bert_output = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = bert_output[1]
        logits = self.fc(pooled_output)
        return logits

In [43]:
i need help just help me im crying so hard. There is no point living now
I am miserable .My life is over now.
The only reason I haven’t killed myself yet is because of my mom and brothers.If I didn’t have them I’d be dead

SyntaxError: invalid syntax (2067087818.py, line 1)